In [3]:
import pandas as pd
from pathlib import Path
import os
os.getcwd()

'/home/hogune/Documents'

In [5]:
base_path = Path("../Downloads/")  # Change to your actual file path
files_info = {
    "Target_list": {"filename": "Target_list.txt", "column": "Acession"},
    "ExploreHT": {"filename": "O_ExploreHT.txt", "column": "Accession"},
    "Reveal": {"filename": "O_Reveal.txt", "column": "Uniprot"},
    "T48panel": {"filename": "O_T48panel.txt", "column": "Protein ID"},
    "T96panel": {"filename": "O_T96Panel.txt", "column": "Protein ID"},
}

In [6]:
# Load data
dataframes = {
    name: pd.read_csv(base_path / info["filename"], sep="\t")
    for name, info in files_info.items()
}

In [8]:
# Extract protein sets
#protein_sets = {
#    name: set(df[info["column"]].dropna().astype(str).str.strip())
#    for name, (df, info) in zip(dataframes.keys(), [(df, files_info[k]) for k, df in dataframes.items()])
#}

In [17]:
# Compute overlaps
#target_ids = protein_sets["Target_list"]
#overlap_results = {
#    name: target_ids.intersection(protein_sets[name])
#    for name in protein_sets if name != "Target_list"
#}
target_df = dataframes["Target_list"]
target_id_col = files_info["Target_list"]["column"]
target_gene_map = dict(zip(target_df[target_id_col], target_df["gene name"]))
target_ids = set(target_gene_map.keys())


In [18]:
# Save overlaps to text files
output_path = base_path / "overlap_outputs"
output_path.mkdir(exist_ok=True)
summary = []

for panel_name, info in files_info.items():
    if panel_name == "Target_list":
        continue
    df = dataframes[panel_name]
    ids_in_panel = set(df[info["column"]].astype(str).str.strip())
    overlap = target_ids.intersection(ids_in_panel)

    # Create DataFrame of overlaps with gene names
    overlap_df = pd.DataFrame({
        "Accession": sorted(overlap),
        "Gene name": [target_gene_map[pid] for pid in sorted(overlap)]
    })

    # Save to file
    output_file = output_path / f"Overlap_Target_vs_{panel_name}.txt"
    overlap_df.to_csv(output_file, sep="\t", index=False)

    # Record summary
    summary.append({
        "Panel": panel_name,
        "Overlap Count": len(overlap)
    })

#for panel, overlap in overlap_results.items():
#    filename = output_path / f"Overlap_Target_vs_{panel}.txt"
#    #with open(filename, "w") as f:
#    #    f.write("\n".join(sorted(overlap)))


In [13]:
# Save overlaps to text files
#output_path = base_path / "overlap_outputs"
#output_path.mkdir(exist_ok=True)

#for panel, overlap in overlap_results.items():
#    filename = output_path / f"Overlap_Target_vs_{panel}.txt"
#    with open(filename, "w") as f:
#        f.write("\n".join(sorted(overlap)))


In [19]:
# Create summary table
#summary_df = pd.DataFrame({
#    "Panel": list(overlap_results.keys()),
#    "Overlap Count": [len(overlap) for overlap in overlap_results.values()]
#})
summary_df = pd.DataFrame(summary)
summary_df.to_csv(output_path / "Overlap_Summary.csv", index=False)

summary_df

,Panel,Overlap Count
0,ExploreHT,461
1,Reveal,150
2,T48panel,59
3,T96panel,253


In [16]:

# Optionally, save summary as CSV
summary_df.to_csv(output_path / "Overlap_Summary.csv", index=False)

# Display the summary
print(summary_df)

       Panel  Overlap Count
0  ExploreHT            461
1     Reveal            150
2   T48panel             59
3   T96panel            253
